In [ ]:
import sys
#import path
sys.path.append("/nfs/students/winter-term-2018/project_2/models/research")
from nms_utils import non_maximum_supression

In [ ]:
import numpy as np
import sys, os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from new_test import evaluate
import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
import cv2
from multiprocessing import Process, Manager
%matplotlib inline

category_names = ['Bier', 'Bier Maß', 'Weißbier', 'Cola', 'Wasser', 'Curry-Wurst', 'Weißwein',
                   'A-Schorle', 'Jägermeister', 'Pommes', 'Burger', 'Williamsbirne', 'Alm-Breze', 'Brotzeitkorb',
                   'Käsespätzle']

In [ ]:
"""
With this ipynb we evaluated our models closer. 
1. Plot the precision-recall curves
2. Plot the predicted bounding boxes
"""

# The parameters for the evaluation function
#PATH_TO_FROZEN_GRAPH = '/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/train/ssd_julius_40/frozen_inference_graph.pb'
PATH_TO_FROZEN_GRAPH = '/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/train/rfcn/9/frozen_inference_graph.pb'
#PATH_TO_FROZEN_GRAPH = '/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/train/ssd_julius_mobilefpn_large/frozen_inference_graph.pb'
#PATH_TO_FROZEN_GRAPH = '/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/train/ssd_mobilenet_alex/1/frozen_inference_graph.pb'
GPU = '0'
RESIZE = (300, 300)
BATCH_SIZE = 1
DATA = '/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/val_data.npz'
#DATA = '/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/data_nightmode.npz'
ic_nms_thresh = 1.

In [ ]:
""" This code is copied from the new_test.py script """
def save_inference_for_batch(*args, **kwargs):
    """ Due to a bug in tensorflow the inference might crash.
    This function evaluates the function run_inference_for_batch until it succeeds """
    ret_dict = Manager().dict()
    def run():
        try:
            ret = run_inference_for_batch(*args, **kwargs)
            ret_dict['ret'] = ret
        except:
            print('Exception during evaluation, retrying')
    while not 'ret' in ret_dict:
        p = Process(target=run)
        p.start()
        p.join()
    return ret_dict['ret']


def run_inference_for_batch(images, graph, batch_size):
    """
    this function runs inference on the images i.e. predicts the bounding boxes, etc.
    images: numpy array of images
    graph: the graph into which the model was loaded
    batch_size: the whole test set does not fit into memory
    returns the bounding boxes, the detected classes and the confidence scores for the images
    """
    with graph.as_default():
        with tf.Session() as sess:
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in ['detection_scores','detection_classes', 'detection_boxes']:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
            ods = []
            for i in range(0, len(images), batch_size):
                ods.append(sess.run(tensor_dict, feed_dict={image_tensor: images[i:i+batch_size]}))
            output_dict = {k: np.concatenate([t[k] for t in ods]) for k in ods[0].keys()}
            output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int32) - 1
            return output_dict

def flatten_out_dict(output_dict, masks=None):
    """ This function removes the first dimension from the arrays returned from run_inference_for_batch
    if masks is not None it applies the masks (entries are removed because of e.g. non-maximum suppression) """
    if masks is None:
            return np.array(list(map(len, output_dict['detection_scores']))), output_dict['detection_scores'].flatten(), output_dict['detection_classes'].astype(np.int32).flatten()
    fmasks = masks.flatten()
    return np.array(list(map(sum, masks))), output_dict['detection_scores'].flatten()[fmasks], output_dict['detection_classes'].astype(np.int32).flatten()[fmasks]

In [ ]:
# load the model into detection_graph
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU)
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
# load the test set and its labels
# limgs are the images i.e. this array is of shape (num_images, h, w, 3)
# gt contains the ground truth i.e. is of shape (num_images, num_classes)
# lmi is the x position of the left most bounding box. This is important to ensure that all objects of interest
# are still on the image when the aspect ratio is changed (e.g. from 16:9 to 1:1)
data = np.load(DATA)
limgs, gt, lmi = data['imgs'], data['gt'], data['lmi']
gt_copy = np.copy(gt)

In [ ]:
# load and preprocess the test set images
w, h = RESIZE
imgs = np.zeros((len(limgs), h, w, 3), dtype=np.float32)
for i in range(len(imgs)):
    nw = int(limgs[i].shape[0] / h * w)
    m = min(limgs[i].shape[1] - nw, lmi[i])
    t = limgs[i, :, m:m+nw, :]
    imgs[i] = cv2.resize(t, (w, h))

# run inference i.e. predict the bounding boxes, etc.
out_dict = save_inference_for_batch(imgs, detection_graph, BATCH_SIZE)
# apply interclass non-maximum suppression
masks = None
if ic_nms_thresh < 1:
    masks = np.array([non_maximum_supression(out_dict['detection_boxes'][i], out_dict['detection_scores'][i], iou_thresh=ic_nms_thresh) \
         for i in range(len(out_dict['detection_boxes']))])

# n references the image i.e. the ith bounding box belongs to the n[i]th image
# s is the confidence score
# c is the class prediction
n, s, c = flatten_out_dict(out_dict, masks)
n = np.repeat(np.arange(len(n)), n)

# sort the predictions by their confidence score
sort_indices = np.argsort(s)[::-1]
n, s, c = n[sort_indices], s[sort_indices], c[sort_indices]

In [ ]:
# recall-precision curve
rpg = np.empty((len(n)+2, 4))  # [recall, precision, threshhold, class]
tp, fp, fn = 0, 0, np.sum(gt)

# calculates the precision-recall curve
for i in range(len(n)):
    if gt[n[i]][c[i]] > 0:
        # if detection is correct
        tp += 1
        fn -= 1
        gt[n[i], c[i]] -= 1
    else:
        # if detection is incorrect
        fp += 1
    rpg[i] = tp / (tp + fn), tp / (tp + fp), s[i], c[i]

# ensure that the precision recall curve goes from 0 to 1
rpg[-2:, :] = [rpg[-3, 0]+1e-8, 0, 0, None], [1., 0., 0, None]
# smooth the precision recall curve
rpg[:, 1] = [np.max(rpg[i:, 1]) for i in range(len(rpg))]
# calculate the area under the curve
arpg = np.trapz(y=rpg[:, 1], x=rpg[:, 0])

In [ ]:
# plot the precision-recall curve
plt.plot(rpg[:, 0], rpg[:, 1])
plt.title('AUC: %.2f' % arpg)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.axis([-0.01, 1.01, -.05, 1.05])
#plt.savefig('/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/result_figures/acc_rpg.png', dpi=150)
plt.show()

In [ ]:
# print precision at specific recall thresholds
print(arpg)
print('precision@<accuracy>: <precision> | <threshhold>')
for r in np.arange(.05, 1, .05):
    c = rpg[rpg[:, 0] > r]
    p = c[0, :] if len(c) > 0 else [0., 0., 0., None]
    print('precision@%.2f: %.4f | %.4f' % (r, p[1], p[2]))
for r in np.arange(.85, 1, .01):
    c = rpg[rpg[:, 0] > r]
    p = c[0, :] if len(c) > 0 else [0., 0., 0., None]
    print('precision@%.2f: %.4f | %.4f' % (r, p[1], p[2]))

In [ ]:
# print precision at specific recall
r = .93 # recall
c = rpg[rpg[:, 0] > r]
p = c[0, :] if len(c) > 0 else [0., 0., 0., None] # precision
print('precision@%.2f: %.4f | %.4f' % (r, p[1], p[2]))

In [ ]:
# plot the bounding boxes
p = 0
thresh = 0.8115  # confidence threshold
for i in range(len(imgs)):
    #if i == 10:
    #    break
    if masks is not None:
        bxs = out_dict['detection_boxes'][i][masks[i]]
        clss = out_dict['detection_classes'][i][masks[i]]
        scr = out_dict['detection_scores'][i][masks[i]]
    else:
        bxs = out_dict['detection_boxes'][i]
        clss = out_dict['detection_classes'][i]
        scr = out_dict['detection_scores'][i]
    fig, ax = plt.subplots(1, figsize=(10, 10*h/w))
    ax.imshow(limgs[i] / 255)
    print(i)
    pred_classes = [0] * 15
    for j in range(len(bxs)):
        if scr[j] < thresh:
            continue
        pred_classes[int(clss[j])] += 1
        bx, by = bxs[j, 1] * 1080 + lmi[i], bxs[j, 0] * 1080
        bw, bh = (bxs[j, 3] - bxs[j, 1]) * 1080, (bxs[j, 2] - bxs[j, 0]) * 1080
        rect = patches.Rectangle((bx, by), bw, bh,linewidth=2, edgecolor='g',facecolor='none')
        ax.add_patch(rect)
        plt.text(bx, by, '%s: %.0f%%' % (category_names[int(clss[j])], scr[j]*100))
        print('%s: %.4f' % (category_names[int(clss[j])], scr[j]))
    pred_classes = gt_copy[i] - pred_classes
    if all(p == 0 for p in pred_classes):
        plt.title('All labels correct')
        #plt.savefig(f'/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/result_figures/img_rfcn/c_{i}.jpg', bbox_inches='tight', pad_inches=0, dpi=100)
    else: 
        plt.title('pred-gt: ' + ' | '.join(('%s %d' % (category_names[i], -pred_classes[i])) for i in range(len(pred_classes)) if pred_classes[i] != 0))
        #plt.savefig(f'/nfs/students/winter-term-2018/project_2/models/research/object_detection/training_folder_lsml/result_figures/img_rfcn/e_{i}.jpg', bbox_inches='tight', pad_inches=0, dpi=100)
    plt.show()